In [1]:
import torch_geometric as tg

from models.se3_transformer import Se3EquivariantTransformer, SE3EquivariantTransformerInverseRadiusSquared, SE3EquivariantTransformerMixedHeads
from utils.load_md17 import load_md17
import e3nn

import torch
import torch_geometric as tg
import pytorch_lightning as pl
import torchmetrics

from workflows.md17_experiment import MD17TransformerTask
import datetime as dt


In [2]:
b = {'0': 'normal',
     '1': 'inverse'
     }
a = SE3EquivariantTransformerMixedHeads.construct_from_number_of_channels_and_lmax(num_channels=16,
                                                                             l_max=2,
                                                                             num_features=9, # Derived from dataset
                                                                             num_attention_layers=4,
                                                                             num_attention_heads=4,
                                                                             radial_network_hidden_units=32,
                                                                             invariant_dictionary=b
                                                                                   )

C:\Users\niksm\Documents\CodeForUni\venvs\transformer_invariants\lib\site-packages\torch\jit\_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


In [ ]:
### Prototype workflow

In [ ]:
energy_model = Se3EquivariantTransformer.construct_from_number_of_channels_and_lmax(num_channels=16,
                                                                             l_max=2,
                                                                             num_features=9, # Derived from dataset
                                                                             num_attention_layers=4,
                                                                             num_attention_heads=4,
                                                                             radial_network_hidden_units=32
                                                                             )

In [ ]:
batch_size=2
radius=2

data = load_md17(dataset_name='aspirin CCSD', dataset_dir='../real_datasets', radius=radius)

train_dataloader = tg.loader.DataLoader(data['train'], batch_size=batch_size, shuffle=True)
validation_dataloader = tg.loader.DataLoader(data['validation'], batch_size=batch_size)
test_dataloader = tg.loader.DataLoader(data['test'], batch_size=batch_size)

In [ ]:
MD17TransformerTask.compute_energy_normalisers(dataset=data['train'])

In [ ]:
timestamp = dt.datetime.now()
timestamp = dt.datetime.strftime(timestamp, '%Y-%m-%d_%H-%M')

In [ ]:
logger = pl.loggers.CSVLogger(name="my_test", version=timestamp, save_dir='lightning_logs')
# Logs are saved to os.path.join(save_dir, name, version).

trainer = pl.Trainer(max_epochs=1, logger=logger)
task = MD17TransformerTask(energy_model)

In [ ]:
# torch.autograd.set_detect_anomaly(True)

In [ ]:
trainer.fit(task,
            train_dataloader,
            test_dataloader,
            )

In [ ]:
for b in train_dataloader:
    b = b.clone()